<a href="https://colab.research.google.com/github/jansoe/KISchule/blob/main/A7_0_jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7. Transformer

In der heutigen Notebook-Reihe möchten wir Ihnen verschiedene Transformer-Modelle und deren Anwendung näher vorstellen. Wir starten mit Modellen für das Natural Language Processing (NLP). Diese Modelle werden also so trainiert, dass Sie ein möglichst weitreichendes Sprachverständnis erlernen.

## 7.0 NLP mit BERT und SQuAD

In diesem Notebook werden Sie zwei Varianten der NLP-Modellklasse BERT (**B**idirectional **E**ncoder **R**epresentations from **T**ransformers) kennenlernen, die jeweils auf dem Datensatz SQuAD (**S**tanford **Qu**estion **A**nswering **D**ataset) nachtrainiert wurden.

---

**Anmerkung**

Damit Sie sich in der Welt der Deep-Learning-Modelle zurechtfinden können, müssen Sie mit vielen neuen Abkürzungen umgehen, da heutzutage jedes Projekt ein Akronym beansprucht.

### 7.0.0 Vorbereitungen

Vor den Imports muss zunächst die transformers-Bilbiothek in Version 2.4 installiert werden.

In [ ]:
!pip install transformers==2.4

Obwohl wir für das Arbeiten mit Textdaten hautpsächlich das die ML-Bibliothek `PyTorch` in Kombination mit der `transformers`-Bibliothek verwenden, umfasst die Liste der Imports auch ein Modul des Ihnen bekannten ML-Frameworks `tensorflow`. Dieses wird dafür benötigt, den benötigten Datensatz `SQuAD` (Stanford Question Answering Dataset) herunterzuladen.

In [ ]:
from pprint import pprint
import os
%tensorflow_version 2
import tensorflow_datasets as tfds
import torch
import torch.utils.data as datatools
import transformers
from transformers.data.metrics.squad_metrics import compute_predictions_logits, squad_evaluate

Sollten Sie Interesse daran haben, welche Hardware Ihnen von Google zur Verfügung gestellt wird, können Sie die folgende Code-Zelle ausführen.

In [ ]:
torch.cuda.get_device_name()

Sie werden Zugriff auf Ihr Google Drive benötigen, um bei Bedarf Modelle dort abspeichern und wieder laden zu können.

In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

Wählen Sie nun einen geeigneten Ordner innerhalb Ihres Google Drives, in dem Sie Daten und Modelle abspeichern möchten.

 ---

 **Hinweis**

Das Verzeichnis der Variable `base_dir` muss bereits in Ihrem Google Drive existieren! Verwenden Sie `'/content/drive/My Drive'`, wenn Sie keine besondere Ordnerstruktur in Ihrem Google Drive verwenden.



In [ ]:
base_dir = '/content/drive/My Drive/KI-Schule/2 - Textdaten' # dieser Ordner muss in Ihrem Google Drive bereits existieren
new_dir = '/squad'
model_dir = base_dir + new_dir

Sollten dieser Ordner noch nicht existieren, können Sie ihn über den folgenden Befehl erstellen. Bei einem erneuten Ausführen erscheint die Meldung, dass das Verzeichnis bereits existiert.

In [ ]:
!mkdir '{model_dir}'

Mit dem folgenden Befehl können Sie testen, ob das Verzeichnis existiert und welche Dateien und Unterverzeichnisse es enthält. Haben Sie es frisch angelegt, so erzeugt der Befehl keinerlei Ausgabe. Im Laufe dieses und des nächsten Notebooks werden dann verschiedene Dateien und Ordner in dem Verzeichnis angelegt.

In [ ]:
!ls '{model_dir}'

### 7.0.1 Laden des Modells

Legen Sie nun fest, welches Modell mit welchem Tokenizer verwendet werden soll. Dabei können Sie entweder auf ein vortrainiertes Modell aus der Transfomers-Bibiliothek zurückgreifen oder ein von Ihnen nachtrainiertes aus Ihrem Google Drive laden (siehe A7_1).

Beginnen Sie beim ersten Durchlauf mit dem relativ kleinen Modell `distilbert-base-uncased-distilled-squad`. Der zugehörige Download umfasst ca. 265MB. Beim großen BERT sind es bereits 1,34GB.

In [ ]:
model_name_or_path = 'distilbert-base-uncased-distilled-squad'  # first run
# model_name_or_path = 'bert-large-uncased-whole-word-masking-finetuned-squad'  # Aufgabe 7.0.5
# model_name_or_path = model_dir + '/0'  # Aufgabe 7.1.4
# model_name_or_path = model_dir + '/1'  # Aufgabe 7.1.4

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name_or_path)
model = transformers.AutoModelForQuestionAnswering.from_pretrained(model_name_or_path)

### 7.0.2 Qualitative Evaluation

In diesem Abschnitt können Sie die Funktionalität des Modells stichprobenartig testen. Die offiziellen Testbeispiele für den Squad-Datensatz samt einer komfortablen Benutzeroberfläche, die zum Durchstöbern einlädt, finden Sie [hier](https://rajpurkar.github.io/SQuAD-explorer/explore/1.1/dev/).

Wir definieren zunächst die Funktion `evaluate()`, die drei Parameter verwendet - ein Modell, einen Kontext sowie eine Frage. Als Ausgabe errechnet die Funktion die Antwort des Modells bezogen auf die Frage und den zur Verfügung gestellten Kontext.

In [ ]:
#@title
#@markdown Bitte ausführen: Definition der Funktion evaluate().
def evaluate(model, context, question):
    inputs = tokenizer.encode_plus(
        question, 
        context, 
        return_attention_mask = False, 
        return_tensors = 'pt',
        return_token_type_ids = not('distilbert' in model_name_or_path) 
    )

    # fuer jeden Token erhalten wir die Wahrscheinlichkeit dass er Start/Endpunkt der Antwort ist
    start_scores, end_scores = model(**inputs)

    # Wir wandeln unsere Token-Ids in Token um
    all_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].numpy().squeeze())

    # Und setzen die Anwort aus den Token zwischen hoechster Start- und hoechtser Endwahrscheinlichkeit zusammen
    answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])
    # Jetzt muessen wir noch die Wortschnipsel zu Woertern zusammenfuegen
    answer = answer.replace(' ##', '')
    return answer

Zunächst verwenden wir eines der offiziellen Testbeispiele, um die Fähigkeiten des Modells kennenzulernen.

In [ ]:
context = """\
The Rhine (Romansh: Rein, German: Rhein, French: le Rhin, Dutch: Rijn) \
is a European river that begins in the Swiss canton of Graubünden in the southeastern Swiss Alps, \
forms part of the Swiss-Austrian, Swiss-Liechtenstein border, Swiss-German and then the Franco-German border, \
then flows through the Rhineland and eventually empties into the North Sea in the Netherlands. \
The biggest city on the river Rhine is Cologne, Germany with a population of more than 1,050,000 people. \
It is the second-longest river in Central and Western Europe (after the Danube), at about 1,230 km (760 mi),\
[note 2][note 1] with an average discharge of about 2,900 m3/s (100,000 cu ft/s).\
"""

question =  "Where does the Rhine begin?"
# question = "What is the largest city the Rhine runs through? "
# question = "What river is larger than the Rhine?"

evaluate(model=model, context=context, question=question)

### 7.0.3 Aufgabe: Qualitative Bewertung mit eigenem Text
Definieren Sie mindestens einen eigenen Kontext und stellen Sie dem Modell Fragen dazu. Welche Fragen kann das Model gut beantworten und welche eher nicht? 

---

**Hinweis**

Untersuchen Sie nach Möglichkeit auch, inwiefern der Stil von Kontext und Frage im Verhältnis zum ursprünglichen Stil des SQuAD-Datensatzes Auswirkungen auf das Ergebnis hat.

In [ ]:
# Lösung:

### 7.0.4 Quantitative Evaluation

Jetzt verwenden wir den gesamten Testdatensatz von SQuAD, um eine quantitative Aussage treffen zu können, wie gut das jeweilige Modell die Fragen aus den Kontexten heraus beantworten kann.

Hierfür berechnen wir uns die sogenannte [F1-Score](https://en.wikipedia.org/wiki/F1_score). Die zugehörige Berechnung basiert darauf, welcher Anteil der vom Modell geantworteten Tokens in den richtigen (vorgegebenen) Antworten vorkommen (`Precision`) und welchen Anteil die vom Modell korrekt ermittelten Antwort-Tokens an den vorgegebenen Antwort-Tokens haben (`Recall` oder auch `Sensitivity`). Diese beiden Werte werden so miteinander verrechnet, dass das Ergebnis zwischen 0 (schlechter geht nicht) und 1 (perfekt) liegt.

Für die zugehörige Berechnung benötigen wir also die Antworten des Modells auf alle Fragestellungen im Testdatensatz. Diesen müssen wir zunächst herunterladen.

In [ ]:
#@title
#@markdown Bitte ausführen: Testdaten von SQuAD herunterladen und zum Testen vorbereiten.

# Here we define the dataset preprocessing
dataset_parameter = dict(
    doc_stride = 128, # When splitting up a long document into chunks, how much stride to take between chunks.
    max_seq_length = 384, # The maximum total input sequence length after WordPiece tokenization. Sequences longer than this will be truncated
    max_query_length= 64, # The maximum number of tokens for the question. Questions longer than this will be truncated to this length."
    return_dataset="pt", # we generate data in the pytorch format
)

cache_file = os.path.join(model_dir, 'test_cache')
if not os.path.exists(cache_file):

    tfds_examples = tfds.load("squad")

    # prepare samples
    test_examples = transformers.SquadV1Processor().get_examples_from_dataset(tfds_examples, evaluate=True)
    test_features, test_dataset = transformers.squad_convert_examples_to_features(
        examples=test_examples,
        tokenizer=tokenizer,
        is_training=False,
        **dataset_parameter
    )
    torch.save({"features": test_features, "dataset": test_dataset, "examples": test_examples}, cache_file)

else: # load data
    features_and_dataset = torch.load(cache_file)
    test_features, test_dataset, test_examples = (
        features_and_dataset["features"],
        features_and_dataset["dataset"],
        features_and_dataset["examples"],
    )

Für die Netzwerkberechnungen wird PyTorch verwendet. Daher wird zunächst auch ein `DataLoader`-Objekt erstellt, bei dem konfiguriert werden kann, in welcher Form die Daten dem Netzwerk zugänglich gemacht werden.

In [ ]:
# Hier legen wir fest, dass wir immer X Samples (Frage-Antwort-Paare) gleichzeitig vorhersagen.
test_dataloader = datatools.DataLoader(
    test_dataset, 
    # Anzahl der Samples pro Batch, aus Performance-Gründen so viele wie auf die GPU passen.
    # D.h. 6 bei einer T4 oder P4, 12 bei einer P100.
    batch_size = 6
)

Und schließlich verwenden wir den DataLoader, um nach und nach Modellvorhersagen für alle Fragestellungen der Testdaten berechnen zu lassen.

In [ ]:
#device = 'cpu'
device = 'cuda'

_ = model.to(device)

model.eval()
test_results = []

# Für jeden Batch  ...
for step, batch in enumerate(test_dataloader):

    # ... laden wir die Daten auf die GPU/CPU
    batch = tuple(t.to(device) for t in batch)

    # ... bereiten die Inputs entsprechend vor (je nach verwendetem Modell)
    inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
        }
    if "distilbert" not in model_name_or_path:
            inputs["token_type_ids"] = batch[2] 

    # ... und schieben die Daten durch das Netz
    # (zum Testen ohne Gradientenberechnung, um effizienter zu sein)
    with torch.no_grad():
        outputs = model(**inputs)

    example_indices = batch[3]
    for i, example_index in enumerate(example_indices):
        
        feature_id = int(test_features[example_index.item()].unique_id)
        
        # und berechenen den besten Start- und Endpunkt für die Antwort
        output = [output[i].detach().cpu().tolist() for output in outputs]
        start_logits, end_logits = output
        result = transformers.data.processors.squad.SquadResult(feature_id, start_logits, end_logits)
        test_results.append(result)

Aus den Modellvorhesagen müssen nun noch die eigentlichen Antworten generiert werden. Hierfür kann die Methode `compute_predictions_logits()` der Transformers-Bibliothek verwendet werden. Auf die zugehörigen Details gehen wir im Rahmen dieses Notebooks jedoch nicht näher ein und haben die zughörigen Einstellungen der Übersicht halber ausgeblendet.

In [ ]:
#@title
#@markdown Bitte ausführen: Einstellungen für die Modellvorhersage (pred_params)

pred_params = dict(
    n_best_size = 2, # The total number of n-best predictions to generate
    max_answer_length = 30, # The maximum length of an answer that can be generated. This is needed because the start and end predictions are not conditioned on one another."
    do_lower_case = True,
    null_score_diff_threshold = 0,
    tokenizer = tokenizer,
    output_prediction_file = os.path.join(model_dir, "predictions.json"),
    output_nbest_file = os.path.join(model_dir, "nbest_predictions.json"),
    output_null_log_odds_file = None,
    verbose_logging = False,
    version_2_with_negative = False
)

Mit diesen Einstellungen lassen sich nun die eigentlichen Modellantworten bestimmen.

In [ ]:
predictions = compute_predictions_logits(test_examples, test_features, test_results, **pred_params)

Schauen wir uns bevor wir den F1-Score berechnen zunächst ein Beispiel für eine Kombinationen aus Kontext, zughöriger Frage sowie den als korrekt gelabelten Antworten und der Modellvorhersage an. 

In [ ]:
qa_pair = test_examples[3]  # hier können Sie sich ein Beispiel aussuchen

print('== Kontext Text ==')
pprint(qa_pair.context_text)
print('\n== Frage ==')
print(qa_pair.question_text)
print('\n== Als korrekt markierte Antworten ==')
print([a['text'] for a in qa_pair.answers])
print('\n== Vorhergesagte Antwort ==')
print(predictions[qa_pair.qas_id])

Schließlich lässt sich die Funktion `squad_evaluate()` der Transformer-Bibliothek dafür verwenden, um unterschiedliche Bewertungen der Leistungsfähigkeit des Modells berechnen zu lassen. Wir picken uns die recht ausssagekräftige F1-Score heraus.

In [ ]:
scores = squad_evaluate(test_examples, predictions)
print(scores['f1'])

Notieren Sie sich den Score für jedes Modell, für das Sie ihn berechnet haben. Hierfür können Sie das folgende Dictionary verwenden:

In [ ]:
quantitative_results = {
    'distilbert-base-uncased-distilled-squad' : None,  # Erster Durchlauf dieses Notebooks
    'bert-large-uncased-whole-word-masking-finetuned-squad' : None,  # Aufgabe 7.0.5
    'bert-base-uncased' : None,  # Aufgabe 7.1.
}
pprint(quantitative_results)

### 7.0.5 Aufgabe: Das große Modell

Das gestete Modell `distilbert-base-uncased-distilled-squad` ist besonders klein und auf Effizienz getrimmt. Größere Modelle haben längere Laufzeiten, sind aber auch performanter. 

Laden Sie in Abschnitt 7.0.1 das Modell `bert-large-uncased-whole-word-masking-finetuned-squad` und führen Sie ansclhießend die Schritte der Abschnitte 7.0.2 und 7.0.4 für die qualitative und quantitative Evaluation des Modells aus. Welche Unterschiede in der Güte der Ergebnisse beobachten Sie?

<bitte ausfüllen>

---

![insitubytes](https://drive.google.com/uc?id=1EAJK7AI9tcZRo3VvYq7vEKGxk7vmK2Ff)